In [ ]:
here are my new variables :
<<<# 3.‑‑ Variables (all in per‑unit)  –– identical domains
I2_MAX = max(pyo.value(model.I_MAX[t]) for t in model.Types) ** 2

model.i_squared_cond    = pyo.Var(model.Lines, model.Day, model.Time, model.Types, domain=pyo.NonNegativeReals, bounds=(0, I2_MAX))
model.i_squared         = pyo.Var(model.Lines, model.Day, model.Time,              domain=pyo.NonNegativeReals, bounds=(0, I2_MAX))
model.v_squared         = pyo.Var(model.Buses, model.Day, model.Time,              domain=pyo.NonNegativeReals, bounds=(V_MIN_PU**2, V_MAX_PU**2))

model.s_sub_squared     = pyo.Var(model.Substations, model.Day, model.Time, domain=pyo.NonNegativeReals, bounds=(0, model.SG.value**2))


model.p_flow            = pyo.Var(model.Lines, model.Day, model.Time, model.Types, domain=pyo.Reals)
model.q_flow            = pyo.Var(model.Lines, model.Day, model.Time, model.Types, domain=pyo.Reals)
model.p_flow_total      = pyo.Var(model.Lines, model.Day, model.Time,              domain=pyo.Reals)
model.q_flow_total      = pyo.Var(model.Lines, model.Day, model.Time,              domain=pyo.Reals)
if zone_choisie == "15kv":
    model.p_sub             = pyo.Var(model.Substations, model.Day, model.Time, domain=pyo.NonNegativeReals) #NonNegative
    model.q_sub             = pyo.Var(model.Substations, model.Day, model.Time, domain=pyo.NonNegativeReals) #NonNegative
else:
    model.p_sub             = pyo.Var(model.Substations, model.Day, model.Time, domain=pyo.Reals) #NonNegative
    model.q_sub             = pyo.Var(model.Substations, model.Day, model.Time, domain=pyo.NonNegativeReals) #NonNegative

model.voltage_drop_term = pyo.Var(model.Lines, model.Day, model.Time, domain=pyo.Reals)

model.n_cap             = pyo.Var(model.Buses, domain=pyo.NonNegativeIntegers)
model.q_cap_installed   = pyo.Var(model.Buses, domain=pyo.Binary)

model.y_pos             = pyo.Var(model.Lines, model.Day, model.Time, domain=pyo.Binary)
model.y_neg             = pyo.Var(model.Lines, model.Day, model.Time, domain=pyo.Binary)
model.z                 = pyo.Var(model.Lines, model.Types, domain=pyo.Binary)
model.w                 = pyo.Var(model.Substations, domain=pyo.Binary)

if storage_flag:
    # adding storage
    model.p_charge          = pyo.Var(model.Storages, model.Day, model.Time, domain=pyo.NonNegativeReals)
    model.p_discharge       = pyo.Var(model.Storages, model.Day, model.Time, domain=pyo.NonNegativeReals)
    model.state             = pyo.Var(model.Storages, model.Day, model.Time, domain=pyo.Reals)>>>

I want you to adapt my constraints to the new set day. BUT you will not change the way everything is written, all you are allowed to, is add the change to have the constraints that accounts for the days, when needed. I do not want any new constraint, or any constraint that disappear, i do not want to see a new way of writting my constraint. Here are the constraints :
<<<
# 4.‑‑ Constraints  (unchanged equations – they now use pu params)
# Useful maps of incoming / outgoing lines
outgoing_lines_by_bus = defaultdict(list)
incoming_lines_by_bus = defaultdict(list)
for (fb, tb) in model.Lines:
    outgoing_lines_by_bus[fb].append((fb, tb))
    incoming_lines_by_bus[tb].append((fb, tb))


# Active power balance ----------------------------------------------------------

def active_power_balance_rule(m, b, t):
    in_sum = sum(m.p_flow[(l, t, typ)] for l in incoming_lines_by_bus[b] for typ in m.Types)
    out_sum = sum(
        m.p_flow[(l, t, typ)] + m.R[typ] * m.LENGTH[l] * m.i_squared_cond[(l, t, typ)]
        for l in outgoing_lines_by_bus[b] for typ in m.Types
    )
    p_sub_part = m.p_sub[b, t] if b in m.Substations else 0.0
    if storage_flag:
        p_charge_part = m.p_charge[b, t] if b in m.Storages else 0.0
        p_discharge_part = m.p_discharge[b, t] if b in m.Storages else 0.0
        return in_sum - out_sum + p_sub_part + m.P_GEN[b, t] - p_charge_part + p_discharge_part == m.P_DEMAND[b, t]
    else:
        return in_sum - out_sum + p_sub_part + m.P_GEN[b, t]  == m.P_DEMAND[b, t]

model.ActivePowerBalance = pyo.Constraint(model.Buses, model.Time, rule=active_power_balance_rule)

# Reactive power balance --------------------------------------------------------

def reactive_power_balance_rule(m, b, t):
    in_sum = sum(m.q_flow[(l, t, typ)] for l in incoming_lines_by_bus[b] for typ in m.Types)
    out_sum = sum(
        m.q_flow[(l, t, typ)] + m.X[typ] * m.LENGTH[l] * m.i_squared_cond[(l, t, typ)]
        for l in outgoing_lines_by_bus[b] for typ in m.Types
    )
    q_sub_part = m.q_sub[b, t] if b in m.Substations else 0.0
    return in_sum - out_sum + q_sub_part + m.Q_PER_CAP * m.n_cap[b] == m.Q_DEMAND[b, t]

model.ReactivePowerBalance = pyo.Constraint(model.Buses, model.Time, rule=reactive_power_balance_rule)

# Voltage drop per line & time --------------------------------------------------

def voltage_drop_per_t_rule(m, fb, tb, t):
    v_from = m.v_squared[fb, t]
    v_to   = m.v_squared[tb, t]
    drop_expr = sum(
        2 * (m.R[typ] * m.p_flow[((fb, tb), t, typ)] + m.X[typ] * m.q_flow[((fb, tb), t, typ)]) * m.LENGTH[(fb, tb)]
        + m.Z[typ] * (m.LENGTH[(fb, tb)] ** 2) * m.i_squared_cond[((fb, tb), t, typ)]
        for typ in m.Types
    )
    return v_from - v_to == m.voltage_drop_term[(fb, tb), t] + drop_expr

model.VoltageDrop = pyo.Constraint(model.Lines, model.Time, rule=voltage_drop_per_t_rule)


# --- Voltage bounds ---
def voltage_min_rule(m, b, t):
    return m.v_squared[b, t] >= m.V_MIN**2

def voltage_max_rule(m, b, t):
    return m.v_squared[b, t] <= m.V_MAX**2

model.VoltageMin = pyo.Constraint(model.Buses, model.Time, rule=voltage_min_rule)
model.VoltageMax = pyo.Constraint(model.Buses, model.Time, rule=voltage_max_rule)

# --- Voltage drop limit using big-M ---
def voltage_drop_term_pos_rule(m, fb, tb, t):
    return m.voltage_drop_term[(fb, tb),t] <= m.B_MAX * (-(m.y_pos[(fb, tb), t] + m.y_neg[(fb, tb), t] - 1))

def voltage_drop_term_neg_rule(m, fb, tb, t):
    return m.voltage_drop_term[(fb, tb),t] >= -m.B_MAX * (-(m.y_pos[(fb, tb), t] + m.y_neg[(fb, tb), t] - 1))

model.VoltageDropTermPos = pyo.Constraint(model.Lines, model.Time, rule=voltage_drop_term_pos_rule)
model.VoltageDropTermNeg = pyo.Constraint(model.Lines, model.Time, rule=voltage_drop_term_neg_rule)



# --- Current limit with active line ---
def current_limit_active_rule(m, fb, tb, t, typ):
    return m.i_squared_cond[( (fb, tb), t, typ )] <= m.I_MAX[typ]**2 * (m.y_pos[(fb, tb), t] + m.y_neg[(fb, tb), t])

model.CurrentLimitActive = pyo.Constraint(model.Lines, model.Time, model.Types, rule=current_limit_active_rule)

# --- Current limit with selected conductor ---
def current_limit_conductor_rule(m, fb, tb, t, typ):
    return m.i_squared_cond[( (fb, tb), t, typ )] <= m.I_MAX[typ]**2 * m.z[(fb, tb), typ]

model.CurrentLimitConductor = pyo.Constraint(model.Lines, model.Time, model.Types, rule=current_limit_conductor_rule)

# Correct SOC (convex) -----------------------------------------------------------
def soc_apparent_power_rule(m, fb, tb, t):
    return m.p_flow_total[(fb, tb), t] ** 2 + m.q_flow_total[(fb, tb), t] ** 2 \
           <= m.i_squared[(fb, tb), t] * (m.v_squared[tb, t]+m.v_squared[fb, t])/2 

model.ApparentSOC = pyo.Constraint(model.Lines, model.Time, rule=soc_apparent_power_rule)





# --- Power flow bounds with direction ---
def p_flow_dir_rule_1(m, fb, tb, t, typ):
    return m.p_flow[( (fb, tb), t, typ )] <= m.V_MAX * m.I_MAX[typ] * m.y_pos[(fb, tb), t]

def p_flow_dir_rule_2(m, fb, tb, t, typ):
    return m.p_flow[( (fb, tb), t, typ )] >= -m.V_MAX * m.I_MAX[typ] * m.y_neg[(fb, tb), t]

def q_flow_dir_rule_1(m, fb, tb, t, typ):
    return m.q_flow[( (fb, tb), t, typ )] <= m.V_MAX * m.I_MAX[typ] * (m.y_pos[(fb, tb), t] + m.y_neg[(fb, tb), t])

def q_flow_dir_rule_2(m, fb, tb, t, typ):
    return m.q_flow[( (fb, tb), t, typ )] >= -m.V_MAX * m.I_MAX[typ] * (m.y_pos[(fb, tb), t] + m.y_neg[(fb, tb), t])

model.pFlowDir1 = pyo.Constraint(model.Lines, model.Time, model.Types, rule=p_flow_dir_rule_1)
model.pFlowDir2 = pyo.Constraint(model.Lines, model.Time, model.Types, rule=p_flow_dir_rule_2)
model.qFlowDir1 = pyo.Constraint(model.Lines, model.Time, model.Types, rule=q_flow_dir_rule_1)
model.qFlowDir2 = pyo.Constraint(model.Lines, model.Time, model.Types, rule=q_flow_dir_rule_2)

# --- Power flow limit with conductor ---
def p_flow_cond_rule_1(m, fb, tb, t, typ):
    return m.p_flow[( (fb, tb), t, typ )] <= m.V_MAX * m.I_MAX[typ] * m.z[(fb, tb), typ]

def p_flow_cond_rule_2(m, fb, tb, t, typ):
    return m.p_flow[( (fb, tb), t, typ )] >= -m.V_MAX * m.I_MAX[typ] * m.z[(fb, tb), typ]

def q_flow_cond_rule_1(m, fb, tb, t, typ):
    return m.q_flow[( (fb, tb), t, typ )] <= m.V_MAX * m.I_MAX[typ] * m.z[(fb, tb), typ]

def q_flow_cond_rule_2(m, fb, tb, t, typ):
    return m.q_flow[( (fb, tb), t, typ )] >= -m.V_MAX * m.I_MAX[typ] * m.z[(fb, tb), typ]

model.pFlowCond1 = pyo.Constraint(model.Lines, model.Time, model.Types, rule=p_flow_cond_rule_1)
model.pFlowCond2 = pyo.Constraint(model.Lines, model.Time, model.Types, rule=p_flow_cond_rule_2)
model.qFlowCond1 = pyo.Constraint(model.Lines, model.Time, model.Types, rule=q_flow_cond_rule_1)
model.qFlowCond2 = pyo.Constraint(model.Lines, model.Time, model.Types, rule=q_flow_cond_rule_2)

# --- Substation apparent power constraint ---
def s_sub_squared_min_rule(m, s, t):
    return m.s_sub_squared[s, t] >= m.p_sub[s, t]**2 + m.q_sub[s, t]**2

model.SubstationApparentMin = pyo.Constraint(model.Substations, model.Time, rule=s_sub_squared_min_rule)

# --- Substation capacity limit ---
def s_sub_squared_cap_rule(m, s, t):
    return m.s_sub_squared[s, t] <= m.SG**2 * m.w[s]

model.SubstationCap = pyo.Constraint(model.Substations, model.Time, rule=s_sub_squared_cap_rule)

# Voltage level = 1 pu at substation
def v_sub_rule(m, s, t):
    return m.v_squared[s, t] == m.V_SUBSTATION
model.SubstationVoltageCap = pyo.Constraint(model.Substations, model.Time, rule=v_sub_rule)


# --- Capacitor constraints ---
def cap_install_rule(m, b):
    return m.n_cap[b] <= m.N_CAP_MAX * m.q_cap_installed[b]

model.CapInstall = pyo.Constraint(model.Buses, rule=cap_install_rule)

def cap_total_rule(m):
    return sum(m.q_cap_installed[b] for b in m.Buses) <= m.N_CAP_TOTAL

model.CapTotal = pyo.Constraint(rule=cap_total_rule)



# --- Total current = sum des conducteurs ---
def total_current_rule(m, fb, tb, t):
    return m.i_squared[(fb, tb), t] == sum(m.i_squared_cond[( (fb, tb), t, typ )] for typ in m.Types)

model.TotalCurrent = pyo.Constraint(model.Lines, model.Time, rule=total_current_rule)

# --- Total active power = sum des conducteurs ---
def total_p_flow_rule(m, fb, tb, t):
    return m.p_flow_total[(fb, tb), t] == sum(m.p_flow[( (fb, tb), t, typ )] for typ in m.Types)

model.TotalPFlow = pyo.Constraint(model.Lines, model.Time, rule=total_p_flow_rule)

# --- Total reactive power = sum des conducteurs ---
def total_q_flow_rule(m, fb, tb, t):
    return m.q_flow_total[(fb, tb), t] == sum(m.q_flow[( (fb, tb), t, typ )] for typ in m.Types)

model.TotalQFlow = pyo.Constraint(model.Lines, model.Time, rule=total_q_flow_rule)


# --- One direction only per line ---
def one_direction_rule(m, fb, tb, t):
    return m.y_pos[(fb, tb), t] + m.y_neg[(fb, tb), t] <= 1

model.OneDirection = pyo.Constraint(model.Lines, model.Time, rule=one_direction_rule)

# --- Radiality: total active lines = buses - substations ---
def radiality_rule(m, t):
    return sum(m.y_pos[l, t] + m.y_neg[l, t] for l in m.Lines) \
           == (len(buses) - len(substations))

model.Radiality = pyo.Constraint(model.Time, rule=radiality_rule)


# # --- Unique conductor per line if line is active ---
# def one_conductor_per_line_rule(model, fb, tb, t):
#     return sum(model.z[(fb, tb), typ] for typ in model.Types) == model.y_pos[(fb, tb),t] + model.y_neg[(fb, tb),t]
    

# model.OneConductorPerLine = pyo.Constraint(model.Lines, model.Time, rule=one_conductor_per_line_rule)


# (a) jamais plus d’un conducteur statique
model.OneConductorPerLine = pyo.Constraint(
    model.Lines,
    rule=lambda m,i,j:
        sum(m.z[i,j,typ] for typ in m.Types) <= 1
)

# (b) activation ⇒ conducteur existant
def link_activation_conductor(m, i, j, t):
    return m.y_pos[i,j,t] + m.y_neg[i,j,t] \
           <= sum(m.z[i,j,typ] for typ in m.Types)

model.LinkActivationConductor = pyo.Constraint(
    model.Lines, model.Time, rule=link_activation_conductor
)
#(a) garantit qu’on ne peut pas construire deux types de câbles différents sur la même liaison.
#(b) empêche la mise en service d’une ligne si aucun conducteur n’a été installé.

if storage_flag:
    # --- Storage Constraints ---
    def storage_state_rule(m, sto, t):
        # turn the Time Set into a list so we can find indices:
        times = list(m.Time)
        idx   = times.index(t)
        if idx == 0:               # first time-step
            return pyo.Constraint.Skip
        t_prev = times[idx-1]      # the actual “previous” label
    
        return (m.state[sto, t] == m.state[sto, t_prev] + m.p_charge[sto, t]   * m.ETA_CHARGE - m.p_discharge[sto, t]/ m.ETA_DISCHARGE )
    
    model.StorageState = pyo.Constraint(model.Storages, model.Time, rule=storage_state_rule)
    
    def charge_max(m, sto, t):
        return m.p_charge[sto, t] <= m.P_CHARGE_MAX
    def discharge_max(m, sto, t):
        return m.p_discharge[sto, t] <= m.P_DISCHARGE_MAX
    model.ChargeMax = pyo.Constraint( model.Storages, model.Time, rule=charge_max)
    model.DischargeMax = pyo.Constraint( model.Storages, model.Time, rule=discharge_max)
    
    def sto_state_max(m, sto, t):
        return m.state[sto, t] <= m.S_MAX
    def sto_state_min(m, sto, t):
        return m.state[sto, t] >= m.S_MIN
    model.StateMax = pyo.Constraint(model.Storages, model.Time, rule=sto_state_max)
    model.StateMin = pyo.Constraint(model.Storages, model.Time, rule=sto_state_min)
    
    def soc_initial_rule(m, sto):
        t0 = m.Time.first()
        return m.state[sto, t0] == m.s_init
    model.soc_initial = pyo.Constraint(model.Storages, rule=soc_initial_rule)
    
    # 2) Cyclic‐wrap constraint: s[i, last_t] == s[i, first_t]
    def soc_cycle_rule(m, sto):
        t0 = m.Time.first()
        tN = m.Time.last()
        return m.state[sto, tN] == m.state[sto, t0]
    model.soc_cycle = pyo.Constraint(model.Storages, rule=soc_cycle_rule)
    >>>